In [1]:
# Imports and all that good stuff

import pandas as pd
import numpy as np
import matplotlib as plt
from geopy.geocoders import Nominatim

%matplotlib inline
from matplotlib import style
style.use('seaborn-white')

In [2]:
def zipcode(num):
    string_num = str(num)
    num = int(float(string_num[0:5]))
    return num

In [3]:
toxic_data_url = "https://data.delaware.gov/resource/u2vy-dniy.csv"
columns = ['geocoded_facility_location','geocoded_facility_location_address','geocoded_facility_location_city','geocoded_facility_location_state','geocoded_facility_location_zip']
toxic_data = pd.read_csv(toxic_data_url, usecols=columns)

In [4]:
school_data_url = "https://data.delaware.gov/resource/tdwa-q9bt.json"
all_columns = pd.read_json(school_data_url).columns.values.tolist() # Get all column names as a list
wanted_columns = ['schoolname', 'schooltype','geocoded_location','street1','street2','city','state','zip'] 
school_data = pd.read_json(school_data_url)
[all_columns.remove(item) for item in wanted_columns] # remove wanted columns from all column list
                                                      # all column list can now be dropped from school_data using pd.drop()
school_data.drop(all_columns, axis=1,inplace=True)    

In [5]:
toxic_data['geocoded_facility_location_zip'].fillna(value=0,inplace=True) # I was getting errors with the zipcode functio
                                                                          # error raise about nan values, even though
                                                                          # I could not find those values. nan's replaced
                                                                          # by 0.
 # apply zipcode function to zipcode column
toxic_data['geocoded_facility_location_zip'] = toxic_data['geocoded_facility_location_zip'].apply(zipcode)           

In [6]:
school_data['zip'] = school_data['zip'].apply(zipcode)

In [7]:
school_data.head()

,city,geocoded_location,schoolname,schooltype,state,street1,street2,zip
0,Camden,"{'human_address': '{""address"":""238 Sorghum Mil...",Allen Frear Elementary School,PUBLIC,DE,238 Sorghum Mill Road,--,19934
1,Magnolia,"{'human_address': '{""address"":""Post Office Box...",J. Ralph McIlvaine Early Childhood Center,PUBLIC,DE,Post Office Box 258,11 East Walnut,19962
2,Dover,"{'human_address': '{""address"":""3100 Hawthorne ...",Major George S. Welch Elementary School,PUBLIC,DE,3100 Hawthorne Drive,--,19901
3,Camden,"{'human_address': '{""address"":""5 Old North Roa...",Kent Elementary Intensive Learning Center,PUBLIC,DE,5 Old North Road,--,19934
4,Dover,"{'human_address': '{""address"":""3874 Upper King...",Nellie Hughes Stokes Elementary School,PUBLIC,DE,3874 Upper King Road,--,19904


In [77]:
# Function to grab geocoded data of schools

def geo(col):
    df_ = []
    for val in col:
        try:
            df_.append((float(val['latitude']),float(val['longitude'])))
        except KeyError:
            df_.append(np.nan)
            pass
    return df_
    
schools_geocode = geo(school_data['geocoded_location'])    

In [78]:
final_school_data = school_data[['schoolname']].copy()
final_school_data['LatLon'] = schools_geocode

In [81]:
final_school_data.dropna()
# This drops 83 schools
# Will have to get their geocode data
# Possibly with a geocode package (geopy)

,schoolname,LatLon
0,Allen Frear Elementary School,"(39.102626, -75.512466)"
3,Kent Elementary Intensive Learning Center,"(39.11888, -75.55001)"
4,Nellie Hughes Stokes Elementary School,"(39.101939, -75.553832)"
5,Star Hill Elementary School,"(39.09982, -75.533325)"
6,W. B. Simpson Elementary School,"(39.11888, -75.55001)"
7,F. Niel Postlethwait Middle School,"(39.098859, -75.512414)"
8,W. Reily Brown Elementary School,"(39.132507, -75.538795)"
9,Fred Fifer III Middle School,"(39.115734, -75.552113)"
10,Caesar Rodney High School,"(39.117836, -75.542046)"
12,John S. Charlton School,"(39.103432, -75.511277)"


In [86]:
len(toxic_data['geocoded_facility_location'].dropna())

510

In [87]:
len(toxic_data['geocoded_facility_location'])

886

In [94]:
toxic_data['geocoded_facility_location'][856]

'POINT (-75.448267 39.807633)'